<a href="https://colab.research.google.com/github/siddtheshah/vc_modeling/blob/master/region_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install cityhash
import cityhash

from copy import deepcopy

     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for cityhash: filename=cityhash-0.2.3.post9-cp36-cp36m-linux_x86_64.whl size=82882 sha256=b90c6101ebe84addf7e401a4e7c23ed36685b91a5175af10fde72ecb2e049e3c
  Stored in directory: /root/.cache/pip/wheels/a4/4d/ae/cf0f1f8058ca2738fe972c69291083a0a06765271fb1047edd
Successfully built cityhash


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
org_info = pd.read_csv('/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/organizations.csv')

In [4]:
import collections

# Filter down the data purely to investees
org_info = org_info[org_info['primary_role'] == 'company']

# Select organization data
# Clean the df

org_info_selected = org_info[['uuid', 'region']]
org_info_selected = org_info_selected.dropna()
org_info_selected = org_info_selected.reset_index(drop=True)

regions = collections.Counter()

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  if iter == test_stop:
    break
  iter += 1
  if iter % 100000 == 0:
    print(iter, " of ", total_rows)

  region = row['region']  
  regions[region] += 1

# Filter low count regions
low_count_threshold = 15
keys_to_remove = []
for region, count in regions.items():
  if count < low_count_threshold:
    keys_to_remove.append(region)

for key in keys_to_remove:
  del regions[key]

# Create index to word mapping
region_to_index = {}
# Start at 1 so we can put hashed uuid at 0.
region_index = 1
for region, count in regions.items():
  region_to_index[region] = region_index
  region_index += 1

print(region_to_index)
region_size = region_index

Processing  842699  rows.
100000  of  842699
200000  of  842699
300000  of  842699
400000  of  842699
500000  of  842699
600000  of  842699
700000  of  842699
800000  of  842699
{'New York': 0, 'California': 1, 'Illinois': 2, 'Hong Kong Island': 3, 'Dublin': 4, 'Virginia': 5, 'Zurich': 6, 'Texas': 7, 'Florida': 8, 'Arizona': 9, 'Tel Aviv': 10, 'Connecticut': 11, 'Leeds': 12, 'Oxfordshire': 13, 'Washington': 14, 'Luxembourg': 15, 'British Columbia': 16, 'Moscow City': 17, 'Berlin': 18, 'North Carolina': 19, 'Ile-de-France': 20, 'Southern Finland': 21, 'Ontario': 22, 'England': 23, 'Nevada': 24, 'Georgia': 25, 'Colorado': 26, 'Massachusetts': 27, 'Pennsylvania': 28, 'Geneve': 29, 'Goa': 30, 'Tennessee': 31, "Seoul-t'ukpyolsi": 32, 'Sachsen': 33, 'Oklahoma': 34, 'Alabama': 35, 'Central Region': 36, 'Stockholms Lan': 37, 'Ohio': 38, 'Tokyo': 39, 'Zuid-Holland': 40, 'Karnataka': 41, 'Baden-Wurttemberg': 42, 'Surrey': 43, 'Minnesota': 44, 'Maryland': 45, 'New South Wales': 46, 'Michigan': 47

In [5]:
# Print some info about our top regions

print(regions.most_common())

[('California', 82659), ('New York', 36883), ('England', 28046), ('Texas', 23375), ('Florida', 19495), ('Ontario', 16690), ('Massachusetts', 15068), ('Illinois', 14298), ('Ile-de-France', 11933), ('Tokyo', 10536), ('Pennsylvania', 10094), ('Maharashtra', 9325), ('Washington', 9143), ('Georgia', 9122), ('Colorado', 8962), ('New Jersey', 8499), ('Virginia', 8101), ('Ohio', 7687), ('Karnataka', 7365), ('Central Region', 7294), ('Bayern', 6972), ('British Columbia', 6936), ('Delhi', 6749), ('Nordrhein-Westfalen', 6718), ('New South Wales', 6622), ('North Carolina', 6548), ('Madrid', 6428), ('Beijing', 6328), ('Michigan', 6109), ('Catalonia', 5836), ('Arizona', 5814), ('Maryland', 5609), ('Quebec', 5582), ('Berlin', 5546), ('Sao Paulo', 5321), ('Minnesota', 5258), ('Tel Aviv', 5091), ('Noord-Holland', 4979), ('Victoria', 4966), ("Seoul-t'ukpyolsi", 4846), ('Guangdong', 4784), ('Baden-Wurttemberg', 4579), ('Alberta', 4536), ('Zurich', 4291), ('Stockholms Lan', 4282), ('Tennessee', 4143), ('S

In [6]:
from scipy.sparse import coo_matrix
import collections

# Load our filtered region dictionary if we've saved it on disk.

row_indices = []
col_indices = []
data_values = []

iter = 0
test_stop = -1
total_rows = np.shape(org_info_selected)[0]
print("Processing ", total_rows, " rows.")
for (i, row) in org_info_selected.iterrows():
  value = row['region']
  if value in regions:
    hash = cityhash.CityHash64(row['uuid'])

    row_indices.append(i)
    col_indices.append(0)
    data_values.append(hash)

    row_indices.append(i)
    col_indices.append(region_to_index[value])
    data_values.append(1)

unclustered = coo_matrix((data_values, (row_indices, col_indices)))
print(unclustered.toarray().shape)

Processing  842699  rows.
(842699, 1031)


In [17]:
import scipy.sparse

companies = org_info_selected['uuid']

lookup_index = []
lookup_value = []

used_hashes = set()

row_indices = []
col_indices = []
data_values = []
for (i, row) in companies.iteritems():
  uuid = row
  hash = cityhash.CityHash64(uuid)

  if hash in used_hashes:  # Hash collision!
    print(hash)
    continue          # which shouldn't really happen
  used_hashes.add(hash)

  lookup_value.append(uuid)
  lookup_index.append(hash)

  row_indices.append(i)
  col_indices.append(0)
  data_values.append(hash)

hashed_uuids = coo_matrix((data_values, (row_indices, col_indices)))
print(np.shape(unclustered))
print(np.shape(hashed_uuids))
joined = scipy.sparse.hstack([hashed_uuids, unclustered])
  
## Generate company lookup
lookup = pd.DataFrame.from_dict({'uuid':lookup_index, 'hash':data_values})
lookup.to_csv('/lookup.csv')

# category_features = pd.DataFrame.sparse.from_spmatrix(joined)
# category_features.to_pickle('/region_features.pkl')

scipy.sparse.save_npz('/region_features.npz', joined)


(842699, 1031)
(842699, 1)


In [16]:
print(joined)

  (0, 0)	1.3685534557686295e+19
  (1, 0)	7.640156219293676e+17
  (2, 0)	1.0846552445983457e+19
  (3, 0)	5.087506707876194e+18
  (4, 0)	9.094535307341386e+18
  (5, 0)	1.7418110891307706e+19
  (6, 0)	7.773244737896805e+18
  (7, 0)	1.5263381640365195e+19
  (8, 0)	2.7762472306795356e+18
  (9, 0)	1.7179139454557108e+19
  (10, 0)	3.143841549007344e+18
  (11, 0)	1.0303043534068977e+18
  (12, 0)	2.4974301446054984e+18
  (13, 0)	5.507971386325512e+18
  (14, 0)	2.8624645959363738e+17
  (15, 0)	1.22265356693808e+19
  (16, 0)	1.1228059662374996e+19
  (17, 0)	1.1671402361252221e+18
  (18, 0)	5.369011697229158e+18
  (19, 0)	1.4750419745579184e+19
  (20, 0)	6.888766334922708e+18
  (21, 0)	1.106971057344432e+19
  (22, 0)	1.615128531752331e+19
  (23, 0)	8.997274164534324e+17
  (24, 0)	1.569457398809743e+19
  :	:
  (842674, 9)	1.0
  (842675, 3)	1.0
  (842676, 17)	1.0
  (842677, 15)	1.0
  (842678, 26)	1.0
  (842679, 72)	1.0
  (842680, 10)	1.0
  (842681, 36)	1.0
  (842682, 23)	1.0
  (842683, 10)	1.0
  (84